<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
由 <a href="https://sebastianraschka.com">Sebastian Raschka</a> 撰写的 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> 一书的补充代码<br>
<br>代码仓库：<a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 从零实现字节对编码 (BPE) 分词器——简化版

- 这是一个独立的笔记本，出于教学目的从零实现了流行的字节对编码（BPE）分词算法，该算法被GPT-2至GPT-4、Llama 3等模型采用
- 关于分词用途的更多细节，请参考[第 2 章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)；这个代码部分是用于解释BPE算法的额外内容
- OpenAI用于训练早期GPT模型的原始BPE分词器可以在[这里](https://github.com/openai/gpt-2/blob/master/src/encoder.py)找到
- BPE算法最初由Philip Gage在1994年的"[A New Algorithm for Data Compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)"中描述
- 由于计算性能优势，大多数项目（包括Llama 3）如今使用OpenAI开源的[tiktoken 库](https://github.com/openai/tiktoken)；它可以加载预训练的GPT-2与GPT-4分词器（Llama 3模型也使用GPT-4分词器进行训练）
- 上述实现与本笔记本中的实现的不同之处在于：这里额外提供一个用于训练分词器的函数（仅限于教学目的）
- 还有一个名为[minBPE](https://github.com/karpathy/minbpe)的实现，支持训练，可能更加高效（此处的实现更注重教学）；与`minbpe`相比，本notebook实现还可以加载OpenAI的原始分词器词表和合并规则

**这是一个非常直白的教学实现。[bpe-from-scratch.ipynb](bpe-from-scratch.ipynb)中提供了一个更复杂（但也更难读）的实现，其行为与 tiktoken 保持一致。**

&nbsp;
# 1. 字节对编码 (BPE) 的核心思想

- BPE的核心思想是将文本转换成整数表示（token ID），以便训练大语言模型（参见[第 2 章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/bpe-overview.webp" width="600px">

&nbsp;
## 1.1 位与字节

- 在就BPE算法本身之前，先介绍一下字节的概念
- 思考将文本转换为一个字节数组（正如BPE是"byte pair encoding"的缩写）

In [1]:
text = "This is some text"
byte_ary = bytearray(text, "utf-8")
print(byte_ary)

bytearray(b'This is some text')


- 当对`bytearray`对象调用`list()`时，每个字节都会被视为独立元素，从而得到一个由这些字节值构成的整数列表

In [2]:
ids = list(byte_ary)
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


- 这确实是一种可以将文本转换为token ID表示的模式，可供LLM的嵌入层使用
- 但是，这种做法的缺点是需要为每个字符创建一个ID（对于短文本也会产生大量 ID）
- 也就是说，对于一段17个字符的输入文本，会向LLM提供17个token ID

In [3]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


- 如果使用过LLM，就会知道BPE分词器的词表中，为整个单词或子词分配token ID，而不是每个单个字符
- 例如，GPT-2分词器会将相同的文本（"This is some text"）分成4个token，而非17个：`1212, 318, 617, 2420`
- 可以使用交互式[tiktoken app](https://tiktokenizer.vercel.app/?model=gpt2)或[tiktoken 库](https://github.com/openai/tiktoken)来三重验证：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="600px">

```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# prints [1212, 318, 617, 2420]
```

- 由于一个字节由 8 个位组成，因此一个字节可表示 2<sup>8</sup> = 256 个可能值，范围为 0 至 255
- 可以通过执行`bytearray(range(0, 257))`来验证，会报告`ValueError: byte must be in range(0, 256)`
- BPE分词器通常使用这256个值作为第一批单字符token；可以通过下面的代码直接看到

```python
import tiktoken
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")
"""
prints:
0: !
1: "
2: #
...
255: �? # <---- single character tokens up to here
256:  t
257:  a
...
298: ent
299:  n
"""
```

- 上面的列表中，第256和257项并非单字符值，而是两个字符（空格 + 字母），这是原始GPT-2 BPE分词器的一个小缺点（在GPT-4的分词器中已得到改进）

&nbsp;
## 1.2 构建词表

- BPE分词算法的目标是构建一个由常见子词构成的词表，比如`298: ent`（例如 *entangle, entertain, enter, entrance, entity, ...* 中都可以找到），或者包含完整单词，例如

```
318: is
617: some
1212: This
2420: text
```

- BPE算法最初由Philip Gage在1994年的"[A New Algorithm for Data Compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)"中描述
- 在深入代码实现之前，现代 LLM 分词器使用的形式可以概括如下

&nbsp;
## 1.3 BPE算法概览

**1. 识别常见字节对**
- 在每次迭代中，扫描文本，找到最常出现的两个接连字节（或字符）

**2. 替换并记录**

- 将这对字节用一个新ID代替（不能和现有ID冲突，例如从0...255开始时，第一个新ID例如256）
- 将这个映射记录在查找表中
- 查找表的大小是一个超参数，也就是“词表大小”（GPT-2为50,257）

**3. 重复直至没有新收益**

- 反复执行前两步，持续合并最常出现的组合
- 当无法再次压缩时（例如不再有字组出现多于一次）即停

**解压（解码）**

- 要恢复原始文本，倒序执行替换，将每个ID用查找表中对应的字节对替之即可


&nbsp;
## 1.4 BPE算法示例

### 1.4.1 编码部分（步骤1和2）的实例

- 假设有一段文本（训练数据集）`the cat in the hat`，想用它构建BPE分词器的词表

**Iteration 1**

1. 识别常见字符对
  - 在这段文本中，"th"出现了两次（开头和第二个"e"之前）

2. 替换并记录
  - 用一个未被使用的新token ID代替"th"，比如256
  - 新的文本：`<256>e cat in <256>e hat`
  - 新的词表为

```
  0: ...
  ...
  256: "th"
```

**Iteration 2**

1. **识别常见字符对**  
   - 在`<256>e cat in <256>e hat`中，`<256>e`出现了两次

2. **替换并记录**  
   - 用一个新token ID代替`<256>e`，例如257  
   - 新的文本为：
     ```
     <257> cat in <257> hat
     ```
   - 更新后的词表：
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     ```

**Iteration 3**

1. **识别常见字符对**  
   - 在`<257> cat in <257> hat`中，`<257> `出现了两次（开头和"hat"前）

2. **替换并记录**  
   - 用一个新token ID代替`<257> `，例如258  
   - 新的文本为：
     ```
     <258>cat in <258>hat
     ```
   - 更新后的词表：
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     258: "<257> "
     ```
     
- 以此类推

&nbsp;
### 1.4.2 解码部分（步骤 3）的实例

- 为了恢复原始文本，将每个token ID按引入顺序倒序替换回其对应的字符对
- 以最终的压缩文本`<258>cat in <258>hat`为起点
- 将`<258>`替换回`<257> `：`<257> cat in <257> hat`  
- 将`<257>`替换回`<256>e`：`<256>e cat in <256>e hat`
- 将`<256>`替换回"th"：`the cat in the hat`

&nbsp;
## 2. 一个简单的 BPE 实现

- 下面的实现是一个Python类，模仿`tiktoken`的Python API
- 需要注意，上面的编码部分描述的是通过`train()`进行的原始训练步骤；不过`encode()`方法的工作方式也很相似（只是因为处理特殊token，看起来稍微复杂一些）：

1. 将输入文本拆分为单独字节
2. 逐步寻找并替换（合并）相邻token（字节对），只要它们与已学习的BPE合并规则匹配（按照排名从高到低）
3. 持续合并，直到不能再合并任何token
4. 最终的token ID列表就是编码结果

In [4]:
from collections import Counter, deque
from functools import lru_cache


class BPETokenizerSimple:
    def __init__(self):
        # Maps token_id to token_str (e.g., {11246: "some"})
        self.vocab = {}
        # Maps token_str to token_id (e.g., {"some": 11246})
        self.inverse_vocab = {}
        # Dictionary of BPE merges: {(token_id1, token_id2): merged_token_id}
        self.bpe_merges = {}

    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        Train the BPE tokenizer from scratch.

        Args:
            text (str): The training text.
            vocab_size (int): The desired vocabulary size.
            allowed_special (set): A set of special tokens to include.
        """

        # Preprocess: Replace spaces with 'Ġ'
        # Note that Ġ is a particularity of the GPT-2 BPE implementation
        # E.g., "Hello world" might be tokenized as ["Hello", "Ġworld"]
        # (GPT-4 BPE would tokenize it as ["Hello", " world"])
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)

        # Initialize vocab with unique characters, including 'Ġ' if present
        # Start with the first 256 ASCII characters
        unique_chars = [chr(i) for i in range(256)]

        # Extend unique_chars with characters from processed_text that are not already included
        unique_chars.extend(char for char in sorted(set(processed_text)) if char not in unique_chars)

        # Optionally, ensure 'Ġ' is included if it is relevant to your text processing
        if 'Ġ' not in unique_chars:
            unique_chars.append('Ġ')

        # Now create the vocab and inverse vocab dictionaries
        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # Add allowed special tokens
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # Tokenize the processed_text into token IDs
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE steps 1-3: Repeatedly find and replace frequent pairs
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:  # No more pairs to merge. Stopping training.
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id

        # Build the vocabulary with merged tokens
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id

    def encode(self, text):
        """
        Encode the input text into a list of token IDs.

        Args:
            text (str): The text to encode.

        Returns:
            List[int]: The list of token IDs.
        """
        tokens = []
        # Split text into tokens, keeping newlines intact
        words = text.replace("\n", " \n ").split()  # Ensure '\n' is treated as a separate token

        for i, word in enumerate(words):
            if i > 0 and not word.startswith("\n"):
                tokens.append("Ġ" + word)  # Add 'Ġ' to words that follow a space or newline
            else:
                tokens.append(word)  # Handle first word or standalone '\n'

        token_ids = []
        for token in tokens:
            if token in self.inverse_vocab:
                # token is contained in the vocabulary as is
                token_id = self.inverse_vocab[token]
                token_ids.append(token_id)
            else:
                # Attempt to handle subword tokenization via BPE
                sub_token_ids = self.tokenize_with_bpe(token)
                token_ids.extend(sub_token_ids)

        return token_ids

    def tokenize_with_bpe(self, token):
        """
        Tokenize a single token using BPE merges.

        Args:
            token (str): The token to tokenize.

        Returns:
            List[int]: The list of token IDs after applying BPE.
        """
        # Tokenize the token into individual characters (as initial token IDs)
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocab: {missing_chars}")

        can_merge = True
        while can_merge and len(token_ids) > 1:
            can_merge = False
            new_tokens = []
            i = 0
            while i < len(token_ids) - 1:
                pair = (token_ids[i], token_ids[i + 1])
                if pair in self.bpe_merges:
                    merged_token_id = self.bpe_merges[pair]
                    new_tokens.append(merged_token_id)
                    # Uncomment for educational purposes:
                    # print(f"Merged pair {pair} -> {merged_token_id} ('{self.vocab[merged_token_id]}')")
                    i += 2  # Skip the next token as it's merged
                    can_merge = True
                else:
                    new_tokens.append(token_ids[i])
                    i += 1
            if i < len(token_ids):
                new_tokens.append(token_ids[i])
            token_ids = new_tokens

        return token_ids

    def decode(self, token_ids):
        """
        Decode a list of token IDs back into a string.

        Args:
            token_ids (List[int]): The list of token IDs to decode.

        Returns:
            str: The decoded string.
        """
        decoded_string = ""
        for token_id in token_ids:
            if token_id not in self.vocab:
                raise ValueError(f"Token ID {token_id} not found in vocab.")
            token = self.vocab[token_id]
            if token.startswith("Ġ"):
                # Replace 'Ġ' with a space
                decoded_string += " " + token[1:]
            else:
                decoded_string += token
        return decoded_string

    @lru_cache(maxsize=None)
    def get_special_token_id(self, token):
        return self.inverse_vocab.get(token, None)

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'.")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                # Remove the 2nd token of the pair, 1st was already removed
                dq.popleft()
            else:
                replaced.append(current)

        return replaced


- `BPETokenizerSimple`类的代码较长，详细对每个细节的讨论不在本notebook范围之内，下一节会通过使用概览来帮助我们更好理解各个方法

## 3. BPE实现游览

- 实践上，强烈建议使用[tiktoken](https://github.com/openai/tiktoken)，因为上面的实现主要关注可读性和教学价值，而不是性能
- 然而，其用法与tiktoken大致相同，只是tiktoken没有训练方法
- 下面通过一些例子来看`BPETokenizerSimple` Python代码如何工作（探讨具体代码不在本notebook范围）

### 3.1 训练、编码与解码

- 首先，我们以一段示例文本作为训练数据集：

In [5]:
import os
import urllib.request

if not os.path.exists("../01_main-chapter-code/the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "../01_main-chapter-code/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("../01_main-chapter-code/the-verdict.txt", "r", encoding="utf-8") as f: # added ../01_main-chapter-code/
    text = f.read()

- 接着，以1,000的词表大小初始化并训练BPE分词器
- 注意由于前面提到的字节值，词表默认已经包含255个条目，因此只需要“学习”745个词表项
- 作为对比，GPT-2的词表大小为50,257，GPT-4的词表为 00,256（tiktoken中的`cl100k_base`），而GPT-4o使用199,997（tiktoken中的`o200k_base`）；它们的训练数据都远大于上面的示例文本

In [6]:
tokenizer = BPETokenizerSimple()
tokenizer.train(text, vocab_size=1000, allowed_special={"<|endoftext|>"})

- 可能想查看词表的具体内容（但这会生成很长的列表）

In [7]:
# print(tokenizer.vocab)
print(len(tokenizer.vocab))

1000


- 该词表通过大约742次合并得到（约等于`1000 - len(range(0, 256))`）

In [8]:
print(len(tokenizer.bpe_merges))

742


- 这意味着最前面的256个条目都是单字符token

- 接下来，使用`encode`方法基于这些合并规则对一些文本进行编码

In [9]:
input_text = "Jack embraced beauty through art and life."
token_ids = tokenizer.encode(input_text)
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [10]:
print("Number of characters:", len(input_text))
print("Number of token IDs:", len(token_ids))

Number of characters: 42
Number of token IDs: 20


- 从上述长度可以看到，一句42个字符的句子被编码成20个token ID，相比按字符/字节编码方式，输入长度几乎减半

- 请注意，词表本身会在`decode()`方法中使用，以便将token ID映射回文本

In [11]:
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [12]:
print(tokenizer.decode(token_ids))

Jack embraced beauty through art and life.


- 逐个遍历每个token ID能帮助更好地理解词表是如何完成解码的

In [13]:
for token_id in token_ids:
    print(f"{token_id} -> {tokenizer.decode([token_id])}")

424 -> Jack
256 ->  
654 -> em
531 -> br
302 -> ac
311 -> ed
256 ->  
296 -> be
97 -> a
465 -> ut
121 -> y
595 ->  through
841 ->  ar
116 -> t
287 ->  a
466 -> nd
256 ->  
326 -> li
972 -> fe
46 -> .


- 可以看到，大多数token ID表示2个字符的子词；这是因为训练文本非常短、重复词不多，并且选择了相对较小的词表大小

- 总而言之，调用`decode(encode())`应该能够还原任意输入文本

In [14]:
tokenizer.decode(tokenizer.encode("This is some text."))

'This is some text.'

&nbsp;
# 4. 结论

- 就是这样！这就是BPE的整体工作流程，并包含一个可用于创建新分词器的训练方法 
- 希望这个简短的教程在教学方面对你有所帮助；若有疑问，欢迎在[这里](https://github.com/rasbt/LLMs-from-scratch/discussions/categories/q-a)开启新的讨论

**这是一个非常朴素的教学实现。[bpe-from-scratch.ipynb](bpe-from-scratch.ipynb)提供了一个更复杂（但也更难读）的实现，其行为与tiktoken一致。**